In [1]:
import itertools
import pandas as pd
import collections
import numpy as np
import re
import pickle
from tqdm import tqdm


path_embedding = './data/embedding/'
df_train = pd.read_csv('./data/WikiQACorpus/WikiQA-train.tsv', sep='\t')
df_test = pd.read_csv('./data/WikiQACorpus/WikiQA-test.tsv', sep='\t')
print('Shape_train: ', df_train.shape)
print('Shape_test: ', df_test.shape)
df_train.head(12)

def saveobj(obj, fname):
    with open(fname, 'wb') as fin:
        pickle.dump(obj, fin, pickle.HIGHEST_PROTOCOL)

Shape_train:  (20347, 7)
Shape_test:  (6116, 7)


In [2]:
word2index = {}
index2word = {}
padding_index = 0
padding_vector = [0]
padding_word = '<pad>'
index2vector = {padding_index:padding_vector}
with open(f'{path_embedding}glove.6B.200d.txt', 'rb') as f:
    for i, l in enumerate(tqdm(f)):
        line = l.decode().split()
        word = line[0]
        vect = list(map(float, line[1:]))
        # Так как "0" зарезервирован под <PAD>, =+1 каждому idx.
        idx = i+1
        word2index[word] = idx
        index2word[idx] = word
        index2vector[idx] = vect
        
index2vector[padding_index] = padding_vector*len(vect)
index2word[padding_index] = padding_word
word2index[padding_word] = padding_index
        
assert len(index2vector) == len(index2word) == len(word2index)

400001it [00:30, 13112.82it/s]


In [3]:
sub = r"[^A-Za-z]+"
df_train.loc[:,'Question'] = df_train.loc[:, 'Question'].apply(lambda x: re.sub(sub, ' ', x).strip().lower())
df_train.loc[:,'Sentence'] = df_train.loc[:, 'Sentence'].apply(lambda x: re.sub(sub, ' ', x).strip().lower())
df_test.loc[:,'Question'] = df_test.loc[:, 'Question'].apply(lambda x: re.sub(sub, ' ', x).strip().lower())
df_test.loc[:,'Sentence'] = df_test.loc[:, 'Sentence'].apply(lambda x: re.sub(sub, ' ', x).strip().lower())
df_train.head()

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label
0,Q1,how are glacier caves formed,D1,Glacier cave,D1-0,a partly submerged glacier cave on perito more...,0
1,Q1,how are glacier caves formed,D1,Glacier cave,D1-1,the ice facade is approximately m high,0
2,Q1,how are glacier caves formed,D1,Glacier cave,D1-2,ice formations in the titlis glacier cave,0
3,Q1,how are glacier caves formed,D1,Glacier cave,D1-3,a glacier cave is a cave formed within the ice...,1
4,Q1,how are glacier caves formed,D1,Glacier cave,D1-4,glacier caves are often called ice caves but t...,0


In [4]:
%%time
df_train.loc[:, 'Question_encoded'] = df_train.loc[:, 'Question'].apply(
    lambda x: [word2index.get(i, word2index['<unk>']) for i in x.split()])
df_train.loc[:, 'Sentence_encoded'] = df_train.loc[:, 'Sentence'].apply(
    lambda x: [word2index.get(i, word2index['<unk>']) for i in x.split()])

df_test.loc[:, 'Question_encoded'] = df_test.loc[:, 'Question'].apply(
    lambda x: [word2index.get(i, word2index['<unk>']) for i in x.split()])
df_test.loc[:, 'Sentence_encoded'] = df_test.loc[:, 'Sentence'].apply(
    lambda x: [word2index.get(i, word2index['<unk>']) for i in x.split()])

CPU times: user 1.53 s, sys: 56 ms, total: 1.58 s
Wall time: 1.58 s


In [5]:
df_train.head()

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Question_encoded,Sentence_encoded
0,Q1,how are glacier caves formed,D1,Glacier cave,D1-0,a partly submerged glacier cave on perito more...,0,"[198, 33, 9683, 11375, 1348]","[8, 3547, 14174, 9683, 6543, 14, 153763, 12691..."
1,Q1,how are glacier caves formed,D1,Glacier cave,D1-1,the ice facade is approximately m high,0,"[198, 33, 9683, 11375, 1348]","[1, 1719, 13522, 15, 2128, 1994, 153]"
2,Q1,how are glacier caves formed,D1,Glacier cave,D1-2,ice formations in the titlis glacier cave,0,"[198, 33, 9683, 11375, 1348]","[1719, 14062, 7, 1, 350149, 9683, 6543]"
3,Q1,how are glacier caves formed,D1,Glacier cave,D1-3,a glacier cave is a cave formed within the ice...,1,"[198, 33, 9683, 11375, 1348]","[8, 9683, 6543, 15, 8, 6543, 1348, 426, 1, 171..."
4,Q1,how are glacier caves formed,D1,Glacier cave,D1-4,glacier caves are often called ice caves but t...,0,"[198, 33, 9683, 11375, 1348]","[9683, 11375, 33, 457, 176, 1719, 11375, 35, 3..."


In [6]:
df_test.head()

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Question_encoded,Sentence_encoded
0,Q0,how african americans were immigrated to the us,D0,African immigration to the United States,D0-0,african immigration to the united states refer...,0,"[198, 686, 828, 36, 17017, 5, 1, 96]","[686, 2253, 5, 1, 105, 113, 4666, 5, 2598, 5, ..."
1,Q0,how african americans were immigrated to the us,D0,African immigration to the United States,D0-1,the term african in the scope of this article ...,0,"[198, 686, 828, 36, 17017, 5, 1, 96]","[1, 571, 686, 7, 1, 6174, 4, 38, 1760, 4666, 5..."
2,Q0,how african americans were immigrated to the us,D0,African immigration to the United States,D0-2,from the immigration and nationality act of to...,0,"[198, 686, 828, 36, 17017, 5, 1, 96]","[26, 1, 2253, 6, 9690, 744, 4, 5, 30, 1274, 47..."
3,Q0,how african americans were immigrated to the us,D0,African immigration to the United States,D0-3,african immigrants in the united states come f...,0,"[198, 686, 828, 36, 17017, 5, 1, 96]","[686, 2598, 7, 1, 105, 113, 327, 26, 592, 65, ..."
4,Q0,how african americans were immigrated to the us,D0,African immigration to the United States,D0-4,they include people from different national li...,0,"[198, 686, 828, 36, 17017, 5, 1, 96]","[40, 489, 70, 26, 495, 122, 13545, 1416, 3453,..."


In [7]:
Q_MAXLEN = df_train.Question_encoded.apply(len).max()
S_MAXLEN = df_train.Sentence_encoded.apply(len).max()

def pad_sequence(seq, max_len, padding=0, cut_last=True):
    seq = seq + [padding] * (max_len - len(seq))
    if cut_last:
        seq = seq[:max_len]
    else:
        seq = seq[-max_len:]
    assert len(seq) == max_len
    return np.array(seq)

df_train.loc[:, 'Question_encoded'] = df_train.Question_encoded.apply(lambda x: pad_sequence(x, Q_MAXLEN))
df_train.loc[:, 'Sentence_encoded'] = df_train.Sentence_encoded.apply(lambda x: pad_sequence(x, S_MAXLEN))

df_test.loc[:, 'Question_encoded'] = df_test.Question_encoded.apply(lambda x: pad_sequence(x, Q_MAXLEN))
df_test.loc[:, 'Sentence_encoded'] = df_test.Sentence_encoded.apply(lambda x: pad_sequence(x, S_MAXLEN))

In [8]:
df_train

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Question_encoded,Sentence_encoded
0,Q1,how are glacier caves formed,D1,Glacier cave,D1-0,a partly submerged glacier cave on perito more...,0,"[198, 33, 9683, 11375, 1348, 0, 0, 0, 0, 0, 0,...","[8, 3547, 14174, 9683, 6543, 14, 153763, 12691..."
1,Q1,how are glacier caves formed,D1,Glacier cave,D1-1,the ice facade is approximately m high,0,"[198, 33, 9683, 11375, 1348, 0, 0, 0, 0, 0, 0,...","[1, 1719, 13522, 15, 2128, 1994, 153, 0, 0, 0,..."
2,Q1,how are glacier caves formed,D1,Glacier cave,D1-2,ice formations in the titlis glacier cave,0,"[198, 33, 9683, 11375, 1348, 0, 0, 0, 0, 0, 0,...","[1719, 14062, 7, 1, 350149, 9683, 6543, 0, 0, ..."
3,Q1,how are glacier caves formed,D1,Glacier cave,D1-3,a glacier cave is a cave formed within the ice...,1,"[198, 33, 9683, 11375, 1348, 0, 0, 0, 0, 0, 0,...","[8, 9683, 6543, 15, 8, 6543, 1348, 426, 1, 171..."
4,Q1,how are glacier caves formed,D1,Glacier cave,D1-4,glacier caves are often called ice caves but t...,0,"[198, 33, 9683, 11375, 1348, 0, 0, 0, 0, 0, 0,...","[9683, 11375, 33, 457, 176, 1719, 11375, 35, 3..."
5,Q2,how are the directions of the velocity and for...,D2,Circular motion,D2-0,in physics circular motion is a movement of an...,0,"[198, 33, 1, 6976, 4, 1, 12191, 6, 353, 24027,...","[7, 5788, 9714, 3054, 15, 8, 927, 4, 30, 4600,..."
6,Q2,how are the directions of the velocity and for...,D2,Circular motion,D2-1,it can be uniform with constant angular rate o...,0,"[198, 33, 1, 6976, 4, 1, 12191, 6, 353, 24027,...","[21, 87, 31, 5062, 18, 4191, 20986, 572, 4, 78..."
7,Q2,how are the directions of the velocity and for...,D2,Circular motion,D2-2,the rotation around a fixed axis of a three di...,0,"[198, 33, 1, 6976, 4, 1, 12191, 6, 353, 24027,...","[1, 7831, 205, 8, 3448, 8907, 4, 8, 88, 13970,..."
8,Q2,how are the directions of the velocity and for...,D2,Circular motion,D2-3,the equations of motion describe the movement ...,0,"[198, 33, 1, 6976, 4, 1, 12191, 6, 353, 24027,...","[1, 14042, 4, 3054, 4467, 1, 927, 4, 1, 314, 4..."
9,Q2,how are the directions of the velocity and for...,D2,Circular motion,D2-4,examples of circular motion include an artific...,0,"[198, 33, 1, 6976, 4, 1, 12191, 6, 353, 24027,...","[4104, 4, 9714, 3054, 489, 30, 6706, 2806, 173..."


In [9]:
df_test

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label,Question_encoded,Sentence_encoded
0,Q0,how african americans were immigrated to the us,D0,African immigration to the United States,D0-0,african immigration to the united states refer...,0,"[198, 686, 828, 36, 17017, 5, 1, 96, 0, 0, 0, ...","[686, 2253, 5, 1, 105, 113, 4666, 5, 2598, 5, ..."
1,Q0,how african americans were immigrated to the us,D0,African immigration to the United States,D0-1,the term african in the scope of this article ...,0,"[198, 686, 828, 36, 17017, 5, 1, 96, 0, 0, 0, ...","[1, 571, 686, 7, 1, 6174, 4, 38, 1760, 4666, 5..."
2,Q0,how african americans were immigrated to the us,D0,African immigration to the United States,D0-2,from the immigration and nationality act of to...,0,"[198, 686, 828, 36, 17017, 5, 1, 96, 0, 0, 0, ...","[26, 1, 2253, 6, 9690, 744, 4, 5, 30, 1274, 47..."
3,Q0,how african americans were immigrated to the us,D0,African immigration to the United States,D0-3,african immigrants in the united states come f...,0,"[198, 686, 828, 36, 17017, 5, 1, 96, 0, 0, 0, ...","[686, 2598, 7, 1, 105, 113, 327, 26, 592, 65, ..."
4,Q0,how african americans were immigrated to the us,D0,African immigration to the United States,D0-4,they include people from different national li...,0,"[198, 686, 828, 36, 17017, 5, 1, 96, 0, 0, 0, ...","[40, 489, 70, 26, 495, 122, 13545, 1416, 3453,..."
5,Q0,how african americans were immigrated to the us,D0,African immigration to the United States,D0-5,as such african immigrants are to be distingui...,1,"[198, 686, 828, 36, 17017, 5, 1, 96, 0, 0, 0, ...","[20, 126, 686, 2598, 33, 5, 31, 5471, 26, 686,..."
6,Q3,how large were early jails,D3,Prison,D3-0,a prison from old french prisoun also known as...,0,"[198, 427, 36, 200, 11916, 0, 0, 0, 0, 0, 0, 0...","[8, 998, 26, 168, 349, 400001, 53, 226, 20, 48..."
7,Q3,how large were early jails,D3,Prison,D3-1,imprisonment or incarceration is a legal penal...,0,"[198, 427, 36, 200, 11916, 0, 0, 0, 0, 0, 0, 0...","[7220, 47, 22370, 15, 8, 831, 1878, 13, 108, 3..."
8,Q3,how large were early jails,D3,Prison,D3-2,other terms used are penitentiary correctional...,0,"[198, 427, 36, 200, 11916, 0, 0, 0, 0, 0, 0, 0...","[69, 1080, 181, 33, 23954, 16926, 2456, 41417,..."
9,Q3,how large were early jails,D3,Prison,D3-3,in some legal systems some of these terms have...,0,"[198, 427, 36, 200, 11916, 0, 0, 0, 0, 0, 0, 0...","[7, 78, 831, 1149, 78, 4, 159, 1080, 34, 6132,..."


In [10]:
df_train.to_pickle('./data/processed/wikiqa_df_train.pickle')
df_test.to_pickle('./data/processed/wikiqa_df_test.pickle')

In [11]:
voc_items = {'index2vector':index2vector,
             'word2index':word2index,
             'index2word':index2word}
saveobj(index2vector, './data/processed/index2vector.pickle')
saveobj(word2index, './data/processed/word2index.pickle')
saveobj(index2word, './data/processed/index2word.pickle')

In [12]:
index2vector_arr = pd.DataFrame(index2vector).T.sort_index().values
np.save('./data/processed/index2vector.npy', index2vector_arr)

In [13]:
from scipy.spatial.distance import cosine as cosine_similarity

In [16]:
def token2vector(token):
    return voc_items['index2vector'].get(voc_items['word2index'][token])

token_1 = 'king'
vec_1 = token2vector(token_1)

In [17]:
d = {k:cosine_similarity(vec_1, token2vector(k)) for k in voc_items['word2index'].keys()}

/home/jupyter/anaconda3/envs/pastuhov/lib/python3.6/site-packages/scipy/spatial/distance.py:644: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [18]:
pd.Series(d).sort_values()

king                     0.000000
prince                   0.314543
queen                    0.333480
kingdom                  0.369679
monarch                  0.377565
ii                       0.385356
throne                   0.392529
reign                    0.408832
iii                      0.416287
crown                    0.420352
emperor                  0.444730
uncle                    0.448380
brother                  0.449183
iv                       0.453093
kings                    0.454215
son                      0.457122
henry                    0.460823
ruler                    0.469619
edward                   0.470871
royal                    0.471102
duke                     0.472426
charles                  0.477206
father                   0.480527
nephew                   0.485396
vi                       0.486162
philip                   0.513849
cousin                   0.514001
frederick                0.517867
grandson                 0.519334
vii           